In [40]:
from qdrant_client import QdrantClient, models
from fastembed import TextEmbedding
import numpy as np
import requests
import uuid

## Q1. Embedding the query

Embed the query: 'I just discovered the course. Can I join now?'. Use the 'jinaai/jina-embeddings-v2-small-en' model.
You should get a numpy array of size 512.\
What's the minimal value in this array?

- -0.51
- -0.11
- 0
- 0.51

In [ ]:
embedding_model = TextEmbedding(
    model_name="jinaai/jina-embeddings-v2-small-en")
query = 'I just discovered the course. Can I join now?'
embeddings = list(embedding_model.embed([query]))
query_embedding = embeddings[0]

print(f"Embedding dimension: {query_embedding.shape[0]}")
query_embedding.min()

# answer: -0.11

Embedding dimension: 512


np.float64(-0.11726373551188797)

## Q2. Cosine similarity with another vector

Now let's embed this document:

```python
doc = 'Can I still join the course after the start date?'
```

What's the cosine similarity between the vector for the query and the vector for the document?

In [ ]:
doc = 'Can I still join the course after the start date?'
embedding_doc = list(embedding_model.embed([doc]))
doc_embedding = embedding_doc[0]

In [48]:
dp = doc_embedding.dot(query_embedding)
print(f"cosine similarity: {dp}")

# answer: 0.9

cosine similarity: 0.9008528856818037


## Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:

```python
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]
```

Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.
What's the document index with the highest similarity? (Indexing starts from 0):

In [ ]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
              'section': 'General course-related questions',
              'question': 'Course - Can I still join the course after the start date?',
              'course': 'data-engineering-zoomcamp'},
             {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
              'section': 'General course-related questions',
              'question': 'Course - Can I follow the course after it finishes?',
              'course': 'data-engineering-zoomcamp'},
             {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
              'section': 'General course-related questions',
              'question': 'Course - When will the course start?',
              'course': 'data-engineering-zoomcamp'},
             {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
              'section': 'General course-related questions',
              'question': 'Course - What can I do before the course starts?',
              'course': 'data-engineering-zoomcamp'},
             {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
              'section': 'General course-related questions',
              'question': 'How can we contribute to the course?',
              'course': 'data-engineering-zoomcamp'}]

text_list = []
for doc in documents:
    embedding_document_text = list(embedding_model.embed([doc['text']]))
    text_embedding = embedding_document_text[0]
    dp = text_embedding.dot(query_embedding)
    text_list.append(float(dp))

text_list

# answer: 1

[0.7629684493123693,
 0.8182378325403556,
 0.8085397290762828,
 0.713307860072464,
 0.7304499388234831]

## Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of question and text:

```python
full_text = doc['question'] + ' ' + doc['text']
```

Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?

In [ ]:
embed_list = []

for doc in documents:
    full_text = f"{doc['question']} {doc['text']}"
    embedding_full_text = list(embedding_model.embed([full_text]))
    full_text_embedding = embedding_full_text[0]
    dp = full_text_embedding.dot(query_embedding)
    embed_list.append(float(dp))

embed_list

# answer: 0

[0.8514543121225202,
 0.8436594058703555,
 0.8408287206920106,
 0.7755157711899687,
 0.8086008033213788]

## Q5. Selecting the embedding model

Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?

ANS: 384

## Q6. Indexing with qdrant (2 points)

For the last question, we will use more documents.\
We will select only FAQ records from our ml zoomcamp:

```python
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Add them to qdrant using the model form Q5.\
When adding the data, use both question and answer fields:

```python
text = doc['question'] + ' ' + doc['text']
```

After the data is inserted, use the question from Q1 for querying the collection.\
What's the highest score in the results? (The score for the first returned record):

In [49]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [38]:
qd_client = QdrantClient("http://localhost:6333")

model_name = "BAAI/bge-small-en"
collection_name = 'zoomcamp-hw-2'
EMBEDDING_DIMENSIONALITY = 384

In [39]:
# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [44]:
points = []

for doc in documents:
    text = doc['question'] + ' ' + doc['text']
    point = models.PointStruct(
        id=uuid.uuid4().hex,
        vector=models.Document(text=text, model=model_name),
        payload={
            "text": text
        }  # save all needed metadata fields
    )
    points.append(point)

In [45]:
# insert data to qdrant collection
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files: 100%|██████████| 5/5 [00:04<00:00,  1.01it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [46]:
def search(query, limit=1):
    results = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(  # embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_name
        ),
        limit=limit,  # top closest matches
        with_payload=True  # to get metadata in the results
    )
    return results

In [ ]:
search_result = search(query=query)
search_result

# answer: 0.8703172

QueryResponse(points=[ScoredPoint(id='265b7c25-91df-42b1-bc7c-0ef462bb90bd', version=0, score=0.8703172, payload={'text': 'The course has already started. Can I still join it? Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.'}, vector=None, shard_key=None, order_value=None)])